In [2]:
%matplotlib inline
import starfish

In [31]:
import tifffile
import functools
import json
import os
import re
from typing import Mapping, Tuple, Union

import click
import numpy as np
from slicedimage import ImageFormat

import starfish.core.util.try_import
from starfish.experiment.builder import FetchedTile, TileFetcher, write_experiment_json
from starfish.types import Axes, Coordinates, Features, Number

SHAPE = {Axes.Y: 648, Axes.X: 718}
Z_PLANE_NUM = 78
CHANNEL_NUM = 3
ROUND_NUM = 5

@functools.lru_cache(maxsize=1)
def cached_read_fn(file_path) -> np.ndarray:
    return np.load(file_path)

class SampleDataTile(FetchedTile):

    def __init__(
            self,
            file_path: str,
            z: int
    ) -> None:
        """
        Parameters
        ----------
        file_path : str
            location of the tile
        coordinates : Mapping[Union[str, Coordinates], Union[Number, Tuple[Number, Number]]]
            the coordinates for the selected tile, extracted from the metadata
        z : int
            the z-layer for the selected osmFISH tile
        """
        self.file_path = file_path
        self.z = z
        # dummy coordinates
        self._coordinates = {
            Coordinates.X: (0.0, 0.0001),
            Coordinates.Y: (0.0, 0.0001),
            Coordinates.Z: (0.0, 0.0001),
        }
        
    @property
    def shape(self) -> Mapping[Axes, int]:
        return SHAPE

    @property
    def coordinates(self) -> Mapping[Union[str, Coordinates], Union[Number, Tuple[Number, Number]]]:
        return self._coordinates

    def tile_data(self) -> np.ndarray:
        #return cached_read_fn(self.file_path)[self.z]  # slice out the correct z-plane
        return tifffile.imread(self.file_path)[self.z]

class SampleDataTileFetcher(TileFetcher):
    
    # color_to_num = {'red': 3, 'green': 2, 'blue': 1}
    
    def __init__(self, input_dir: str) -> None:
        self.input_dir = input_dir
        self.num_z = Z_PLANE_NUM  # hard coded for this dataset
        
    def get_tile(self, fov: int, r: int, ch: int, z: int) -> FetchedTile:
        basename = f"round0{r + 1}-ch0{ch + 1}.tif"  # translate to 3d
        file_path = os.path.join(self.input_dir, basename)
        return SampleDataTile(file_path, z)
    
    def generate_codebook(self, output_dir: str) -> None:
        """
        TODO
        Generate and save a codebook from the provided mapping of genes to DNA sequences.
        output_dir : str
            directory in which to save the generated codebook. Codebook is saved as "codebook.json"

        """
        dinucleotides_to_channels = {
            "CT": 3,
            "GT": 2,
            "TT": 1,
            "AG": 3,
            "GG": 1,
            "TG": 2,
            "AC": 2,
            "CC": 1,
            "TC": 3,
            "AA": 1,
            "CA": 2,
            "GA": 3,
        }
        
#         with open(os.path.join(self.input_dir, "genes.csv"), "r") as f:
#             codes = [l.strip().split(",") for l in f.readlines()]  # List[(gene, dna_barcode), ...]

#         def iter_dinucleotides(sequence):
#             i = 0
#             while i + 1 < len(sequence):
#                 yield sequence[i:i + 2]
#                 i += 1

#         # construct codebook target mappings
#         code_array = []
#         for gene, dna_barcode in codes:
#             dna_barcode = dna_barcode[::-1]  # reverse barcode
#             spacetx_barcode = [
#                 {
#                     Axes.ROUND.value: r,
#                     Axes.CH.value: dinucleotides_to_channels[dinucleotide],
#                     Features.CODE_VALUE: 1
#                 } for r, dinucleotide in enumerate(iter_dinucleotides(dna_barcode))
#             ]
#             code_array.append({
#                 Features.CODEWORD: spacetx_barcode,
#                 Features.TARGET: gene
#             })

#         codebook = Codebook.from_code_array(code_array)
#         codebook.to_json(os.path.join(output_dir, "codebook.json"))

class SampleDataDapiTileFetcher(TileFetcher):

    def __init__(self, input_dir: str) -> None:
        """
        The red channel in the TIF files contains staining that labels all cells, 
        which can serve a similar function to DAPI
        """
        self.input_dir = input_dir

    def get_tile(self, fov: int, r: int, ch: int, z: int) -> FetchedTile:
        basename = f"round0{r + 1}-ch03.tif"
        file_path = os.path.join(self.input_dir, basename)
        return StarMapTile(file_path, z)

    
def cli(input_dir, output_dir) -> None:
    """
    TODO
    osmFISH example does not use auxillary images - could try, but need dimensions
    """
    abs_output_dir = os.path.expanduser(output_dir)
    abs_input_dir = os.path.expanduser(input_dir)
    os.makedirs(abs_output_dir, exist_ok=True)

    primary_tile_fetcher = SampleDataTileFetcher(abs_input_dir)

    # This is hardcoded for this example data set
    primary_image_dimensions: Mapping[Union[str, Axes], int] = {
        Axes.ROUND: ROUND_NUM,
        Axes.CH: CHANNEL_NUM,
        Axes.ZPLANE: Z_PLANE_NUM,
    }

    write_experiment_json(
        path=output_dir,
        fov_count=1,
        tile_format=ImageFormat.TIFF,
        primary_image_dimensions=primary_image_dimensions,
        primary_tile_fetcher=primary_tile_fetcher,
        aux_name_to_dimensions={},
        dimension_order=(Axes.ROUND, Axes.CH, Axes.ZPLANE)
    )

    #primary_tile_fetcher.generate_codebook(abs_output_dir)
    
if __name__ == "__main__":
    cli('sample_dat/raw', 'sample_dat/formatted')

/Users/danyfu/.pyenv/versions/3.6.3/lib/python3.6/site-packages/skimage/io/_io.py:141: UserWarning: <_io.BytesIO object at 0x13cc8d830> is a low contrast image
  warn('%s is a low contrast image' % fname)
/Users/danyfu/.pyenv/versions/3.6.3/lib/python3.6/site-packages/skimage/io/_io.py:141: UserWarning: <_io.BytesIO object at 0x13cc7f0a0> is a low contrast image
  warn('%s is a low contrast image' % fname)
/Users/danyfu/.pyenv/versions/3.6.3/lib/python3.6/site-packages/skimage/io/_io.py:141: UserWarning: <_io.BytesIO object at 0x13cc7f0f8> is a low contrast image
  warn('%s is a low contrast image' % fname)
/Users/danyfu/.pyenv/versions/3.6.3/lib/python3.6/site-packages/skimage/io/_io.py:141: UserWarning: <_io.BytesIO object at 0x13cc7f150> is a low contrast image
  warn('%s is a low contrast image' % fname)
/Users/danyfu/.pyenv/versions/3.6.3/lib/python3.6/site-packages/skimage/io/_io.py:141: UserWarning: <_io.BytesIO object at 0x13cc7f200> is a low contrast image
  warn('%s is a low 

/Users/danyfu/.pyenv/versions/3.6.3/lib/python3.6/site-packages/skimage/io/_io.py:141: UserWarning: <_io.BytesIO object at 0x13cc803b8> is a low contrast image
  warn('%s is a low contrast image' % fname)
/Users/danyfu/.pyenv/versions/3.6.3/lib/python3.6/site-packages/skimage/io/_io.py:141: UserWarning: <_io.BytesIO object at 0x13ef9c410> is a low contrast image
  warn('%s is a low contrast image' % fname)
/Users/danyfu/.pyenv/versions/3.6.3/lib/python3.6/site-packages/skimage/io/_io.py:141: UserWarning: <_io.BytesIO object at 0x13ef9c468> is a low contrast image
  warn('%s is a low contrast image' % fname)
/Users/danyfu/.pyenv/versions/3.6.3/lib/python3.6/site-packages/skimage/io/_io.py:141: UserWarning: <_io.BytesIO object at 0x13ef9c3b8> is a low contrast image
  warn('%s is a low contrast image' % fname)
/Users/danyfu/.pyenv/versions/3.6.3/lib/python3.6/site-packages/skimage/io/_io.py:141: UserWarning: <_io.BytesIO object at 0x13ef9c518> is a low contrast image
  warn('%s is a low 

/Users/danyfu/.pyenv/versions/3.6.3/lib/python3.6/site-packages/skimage/io/_io.py:141: UserWarning: <_io.BytesIO object at 0x13cc8f308> is a low contrast image
  warn('%s is a low contrast image' % fname)
/Users/danyfu/.pyenv/versions/3.6.3/lib/python3.6/site-packages/skimage/io/_io.py:141: UserWarning: <_io.BytesIO object at 0x13cc8f3b8> is a low contrast image
  warn('%s is a low contrast image' % fname)
/Users/danyfu/.pyenv/versions/3.6.3/lib/python3.6/site-packages/skimage/io/_io.py:141: UserWarning: <_io.BytesIO object at 0x13cc8f410> is a low contrast image
  warn('%s is a low contrast image' % fname)
/Users/danyfu/.pyenv/versions/3.6.3/lib/python3.6/site-packages/skimage/io/_io.py:141: UserWarning: <_io.BytesIO object at 0x13cc8f468> is a low contrast image
  warn('%s is a low contrast image' % fname)
/Users/danyfu/.pyenv/versions/3.6.3/lib/python3.6/site-packages/skimage/io/_io.py:141: UserWarning: <_io.BytesIO object at 0x13cc8f4c0> is a low contrast image
  warn('%s is a low 

/Users/danyfu/.pyenv/versions/3.6.3/lib/python3.6/site-packages/skimage/io/_io.py:141: UserWarning: <_io.BytesIO object at 0x13ef36308> is a low contrast image
  warn('%s is a low contrast image' % fname)
/Users/danyfu/.pyenv/versions/3.6.3/lib/python3.6/site-packages/skimage/io/_io.py:141: UserWarning: <_io.BytesIO object at 0x13ef363b8> is a low contrast image
  warn('%s is a low contrast image' % fname)
/Users/danyfu/.pyenv/versions/3.6.3/lib/python3.6/site-packages/skimage/io/_io.py:141: UserWarning: <_io.BytesIO object at 0x13ef36410> is a low contrast image
  warn('%s is a low contrast image' % fname)
/Users/danyfu/.pyenv/versions/3.6.3/lib/python3.6/site-packages/skimage/io/_io.py:141: UserWarning: <_io.BytesIO object at 0x13ef36468> is a low contrast image
  warn('%s is a low contrast image' % fname)
/Users/danyfu/.pyenv/versions/3.6.3/lib/python3.6/site-packages/skimage/io/_io.py:141: UserWarning: <_io.BytesIO object at 0x13ef364c0> is a low contrast image
  warn('%s is a low 

/Users/danyfu/.pyenv/versions/3.6.3/lib/python3.6/site-packages/skimage/io/_io.py:141: UserWarning: <_io.BytesIO object at 0x13f01ee08> is a low contrast image
  warn('%s is a low contrast image' % fname)
/Users/danyfu/.pyenv/versions/3.6.3/lib/python3.6/site-packages/skimage/io/_io.py:141: UserWarning: <_io.BytesIO object at 0x13f01ed58> is a low contrast image
  warn('%s is a low contrast image' % fname)
/Users/danyfu/.pyenv/versions/3.6.3/lib/python3.6/site-packages/skimage/io/_io.py:141: UserWarning: <_io.BytesIO object at 0x13f01eeb8> is a low contrast image
  warn('%s is a low contrast image' % fname)
/Users/danyfu/.pyenv/versions/3.6.3/lib/python3.6/site-packages/skimage/io/_io.py:141: UserWarning: <_io.BytesIO object at 0x13f01ef10> is a low contrast image
  warn('%s is a low contrast image' % fname)
/Users/danyfu/.pyenv/versions/3.6.3/lib/python3.6/site-packages/skimage/io/_io.py:141: UserWarning: <_io.BytesIO object at 0x13f01ef68> is a low contrast image
  warn('%s is a low 

In [19]:
ipython = get_ipython()
ipython.magic("gui qt5")

In [32]:
experiment = starfish.Experiment.from_json("formatted/experiment.json")
fov = experiment['fov_000']
image = fov.get_image("primary")
image.sel({Axes.CH: 1, Axes.ROUND: 1})
starfish.display(image)

100%|██████████| 1170/1170 [00:44<00:00, 29.14it/s]
